In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes datasets

In [ ]:
!pip install --pre -U xformers

In [ ]:
!git clone  https://github.com/XZhang97666/AlpaCare.git

In [ ]:
!mv '/content/AlpaCare/data' '/content'

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from tqdm.auto import tqdm
import pandas as pd
import json
from datasets import Dataset
from operator import itemgetter
from os.path import join
from sklearn.model_selection import train_test_split
import numpy as np

# add your token
HF_TOKEN = ''

## Utils

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""



In [ ]:
def read_json(file_name):
    with open(file_name) as f:
        d = json.load(f)
    return d

## Model definition

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
model_name = 'unsloth/gemma-7b-bnb-4bit'
data_name = 'medinstruct-52k'
hf_save_model_name = 'adlbh/gemma-7b-medinstruct-52k'

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)



In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

## MedInstruct-52k

In [ ]:
tokenizer.eos_token

In [ ]:
tokenizer.bos_token

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instruction = examples["instruction"]
    input       = examples["input"]
    output      = examples["output"]
    text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
    return { "text" : text, }

# from datasets import load_dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
def get_train_test_prompts(X_train, X_eval):
    X_train = pd.DataFrame(X_train.apply(formatting_prompts_func, axis=1),
                          columns=["text"])
    X_eval = pd.DataFrame(X_eval.apply(formatting_prompts_func, axis=1),
                          columns=["text"])
    # X_test = pd.DataFrame(X_test.apply(generate_prompt('test'), axis=1), columns=["text"])

    train_data = Dataset.from_pandas(X_train)
    eval_data = Dataset.from_pandas(X_eval)
    # test_data = Dataset.from_pandas(X_test)
    return train_data, eval_data


def get_med_instruct_data():

    med_inst_data = read_json('/content/data/MedInstruct-52k.json')
    print('Alpacare Instruction Data size:', len(med_inst_data))
    print(type(med_inst_data))

    med_inst_train, med_inst_val, y_train, y_val = train_test_split(np.array(range(len(med_inst_data))), [0]*len(med_inst_data), test_size=0.20, random_state=42)
    med_inst_train, med_inst_val = med_inst_train.tolist(), med_inst_val.tolist()
    med_inst_train, med_inst_val = itemgetter(*med_inst_train)(med_inst_data), itemgetter(*med_inst_val)(med_inst_data)

    med_inst_train = pd.DataFrame(med_inst_train)
    med_inst_val = pd.DataFrame(med_inst_val)

    train_data = Dataset.from_pandas(med_inst_train)
    val_data = Dataset.from_pandas(med_inst_val)
    train_data = train_data.map(formatting_prompts_func)
    val_data = val_data.map(formatting_prompts_func)

    print('train size: ', len(med_inst_train))
    print('val size: ', len(med_inst_val))
    print('med_inst_train.columns: ', med_inst_train.columns)
    # train_data, val_data = get_train_test_prompts(med_inst_train, med_inst_val)

    return train_data, val_data



In [ ]:
med_train, med_val = get_med_instruct_data()

In [ ]:
med_train[0]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = med_train,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 1000,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
model.push_to_hub(hf_save_model_name, token = HF_TOKEN) # Online saving
tokenizer.push_to_hub(hf_save_model_name, token = HF_TOKEN) # Online saving

## Upload and Inference

In [ ]:
def infer_model(query, model, tokenizer):
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            query, # instruction
            "", # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    prompt_length = inputs['input_ids'].shape[1]
    outputs = model.generate(**inputs, max_new_tokens = 1000, use_cache = True)
    # answer = tokenizer.batch_decode(outputs)
    answer = tokenizer.decode(outputs[0][prompt_length:])
    return answer



In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = hf_save_model_name, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!



In [ ]:
infer_model('how to treat asthma', model, tokenizer)